<a href="https://colab.research.google.com/github/DiNOV-Tokyo/yolov5/blob/main/DL_torchvsion_ResNet50_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.ResNet50でファインチューニング・推論

参考にしたサイト# https://gurutaka-log.com/pytorch-lightning-fine-tuning



## 1.1 Google Drive をマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/Colab\ Notebooks
#%mkdir yolo_train
%cd yolo_train

/content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks/yolo_train


In [3]:
# pytorch_lightningのインストール
!pip install pytorch_lightning 

     |████████████████████████████████| 915 kB 7.1 MB/s 
     |████████████████████████████████| 118 kB 15.0 MB/s 
     |████████████████████████████████| 829 kB 21.7 MB/s 
     |████████████████████████████████| 10.6 MB 27.6 MB/s 
     |████████████████████████████████| 234 kB 70.1 MB/s 
     |████████████████████████████████| 636 kB 66.0 MB/s 
     |████████████████████████████████| 1.3 MB 52.0 MB/s 
     |████████████████████████████████| 294 kB 68.4 MB/s 
     |████████████████████████████████| 142 kB 68.5 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=19425aefc933bf781c31f94b30a29f28d3a0b099f30bdd3890a1391deb70766c
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstal

## 1.2 モジュール読み込み

In [6]:
# pytorch関係
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import pytorch_lightning as pl
from pytorch_lightning import Trainer

# フォルダ内のデータ読込で使用
from PIL import Image
import glob


## 1.3 データパス設定

In [14]:
# 画像読み取り
fold_path = 'dataset/train/'
imgs = []
for imgs_path in glob.glob(fold_path + '*'):
    imgs.append(glob.glob(imgs_path + '/*'))


In [13]:
print(img_path)

dataset/train/gym


## 1.4 モデル作成　ResNet50

In [8]:
# 学習済みモデル「ResNet50」をインポート
from torchvision.models import resnet50

# 学習済みモデル「ResNet50」をダウンロード
resnet = resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


## 1.5 transform作成

In [17]:
#transform = transforms.Compose(
#    [
#        transforms.Resize(256),  # (256, 256) で切り抜く。
#        transforms.CenterCrop(224),  # 画像の中心に合わせて、(224, 224) で切り抜く
 #       transforms.ToTensor(),  # テンソルにする。
 #       transforms.Normalize(
 #           mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
 #       ),  # 標準化する。
 #   ]
#)


transform = transforms.Compose([
    # 224×224にリサイズ
    transforms.Resize((224, 224)),
    # torch.Tensor型に変換
    transforms.ToTensor(),
    # 学習済みモデルで使われる平均と標準偏差で標準化
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
])

## 1.6 torch.Tensor型に変更

In [23]:
labels = []
img_datas = torch.tensor([])

# 画像データを配列に格納
# torch型に変更
for i,imgs_arr in enumerate(imgs):
    
    for img_path in imgs_arr:
        # ラベルを追加
        labels.append(i)

        # imgデータ（PIL型）の取得
        img = Image.open(img_path)

        # imgデータの前処理
        tensor_img = transform(img)

        # バッチサイズを追加: バッチサイズ, チャンネル, 高さ、幅
        tensor_img = tensor_img.unsqueeze(0)

        # torch.tensorのappend
        img_datas = torch.cat([img_datas, tensor_img],dim=0)

train val に分けて、Dataset準備

In [26]:
# データセット化
datasets = torch.utils.data.TensorDataset(img_datas, labels)

# 訓練:検証 = 85% : 15%
n_train = int(len(datasets) * 0.85)
n_val = len(datasets) - n_train

# 擬似乱数のシード固定
torch.manual_seed(0)

# データセットを訓練用、検証用で分割
train,val = torch.utils.data.random_split(datasets,[n_train,n_val])

AttributeError: ignored

In [22]:
print(labels)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 1.7 モデル学習

In [ ]:
# 訓練用のクラス
class TrainNet(pl.LightningModule):
    @pl.data_loader
    def train_dataloader(self):
        # ミニバッチ
        return torch.utils.data.DataLoader(train, self.batch_size,shuffle=True)
    
    def training_step(self, batch, batch_nb):
        # 入力と目標値を分割
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        results = {'loss': loss}
        return results

# 検証用のクラス
class ValidationNet(pl.LightningModule):

    @pl.data_loader
    def val_dataloader(self):
        return torch.utils.data.DataLoader(val, self.batch_size)

    def validation_step(self, batch, batch_nb):
        x, t = batch
        y = self.forward(x)
        loss = self.lossfun(y, t)
        y_label = torch.argmax(y, dim=1)
        acc = torch.sum(t == y_label) * 1.0 / len(t)
        results = {'val_loss': loss, 'val_acc': acc}
        return results

    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        results =  {'val_loss': avg_loss, 'val_acc': avg_acc}
        return results

# 学習用の全体のクラス
# 訓練用、検証用を継承
class FineTuningNet(TrainNet, ValidationNet):

    # データ数が少ないのでbatch_sizeは大きめ
    def __init__(self, batch_size=256):
        super().__init__()
        self.batch_size = batch_size
        # 畳み込みの部分は学習済みモデルを活用
        self.conv = resnet18(pretrained=True)
        # 全結合層は1000 => 100 => 3
        # 1000はresnet18の仕様
        # 3は分類数（かしゆか、のっち、あーちゃん）
        self.fc1 = nn.Linear(1000, 100)
        self.fc2 = nn.Linear(100, 3)

        # 学習済みのパラメータを固定
        for param in self.conv.parameters():
            param.requires_grad = False

    def lossfun(self, y, t):
        return F.cross_entropy(y, t)

    def configure_optimizers(self):
        return torch.optim.SGD(self.parameters(), lr=0.01)

    def forward(self, x):
        x = self.conv(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
# 再現性の確保
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 学習
fine_net = FineTuningNet()
trainer = Trainer(gpus=1, max_epochs=300)
trainer.fit(fine_net)

メトリックの確認

In [ ]:
trainer.callback_metrics

## 1.8 推論

In [ ]:
# unsqueeze：バッチサイズ1を追加
# cuda:GPUなので必要
x = val[0][0].unsqueeze(0).cuda()

# モデルモードを学習→推論に変更
fine_net.eval()
fine_net.freeze()

F.softmax(fine_net(x)), val[0][1]

## 1.9 モデル保存

In [ ]:
# 学習済みモデルの保存
torch.save(fine_net.state_dict(), 'fine.pt')